In [1]:
from transformers import pipeline

# Replace with your actual HF repo path (e.g., "anujbohra/whisper-marathi-finetuned")
asr_pipeline = pipeline(
    "automatic-speech-recognition",
    model="bohraanuj23/whisper-small-hindi",
    device=0  # set to -1 for CPU
)


config.json:   0%|          | 0.00/1.32k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.86k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

...

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/339 [00:00<?, ?B/s]

Device set to use cuda:0


In [2]:
# audio path from earlier audio extraction
transcription = asr_pipeline("HindiAudioSample.mp3")
# Access the text output
print("Transcribed Text:", transcription['text'])


/usr/local/lib/python3.11/dist-packages/transformers/models/whisper/generation_whisper.py:573: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.


Transcribed Text: उसके लिए दैंक्य। कभि लभी आप अपर्ट बेल देते हो, उसके लिए भी दैंक्य। दो मेले-मेले आपने मेरे चप्पल सिला के लेगा ऐशिक, उसके लिए वी दैंक्य। आप मेरे सारी को भी स्थनी कर देते हो, उसके लिए वी दैंक्य। और यह प्रडो दैंक्य। काय तल रहा पात्वीए? मैंने अपको जाए दी तो अपने मुझे दैंक्यो पोला पाता... तो मेरे तो कित नी दैंक्यो पोल पके... इसलिए अभी पोल रहीं... तैंक्य। सोरी...


In [3]:
import torch
from transformers import WhisperProcessor, WhisperForConditionalGeneration
import torchaudio

# Load processor and model
processor = WhisperProcessor.from_pretrained("bohraanuj23/whisper-small-hindi")
model = WhisperForConditionalGeneration.from_pretrained("bohraanuj23/whisper-small-hindi").to("cuda")

# Load long audio
speech_array, sampling_rate = torchaudio.load("/content/tmkoc.mp3")
speech_array = torchaudio.functional.resample(speech_array, orig_freq=sampling_rate, new_freq=16000).squeeze()

# Parameters
chunk_length_s = 30  # seconds
stride_length_s = 5  # seconds overlap for context
input_stride = stride_length_s * 16000
input_chunk = chunk_length_s * 16000

# Store text
full_transcript = ""

# Process in chunks
start = 0
while start < len(speech_array):
    end = min(start + input_chunk, len(speech_array))
    chunk = speech_array[start:end]

    inputs = processor.feature_extractor(chunk, sampling_rate=16000, return_tensors="pt").input_features.to("cuda")

    with torch.no_grad():
        predicted_ids = model.generate(inputs)
    text = processor.tokenizer.batch_decode(predicted_ids, skip_special_tokens=True)[0]
    full_transcript += " " + text

    start += input_chunk - input_stride  # move window with overlap

print(full_transcript)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


 उसके लिए थैंक्य। कभी-कभी आप अपर्ट बेल देते हो, उसके लिए भी थैंक्य। दो मेले-मेले आपने मेरे चप्पल सिला के लेगा ऐ थे, उसके लिए वी थैंक्य। आप मेरे सारी को भी स्थनी कर देते हो, उसके लिए वी थैंक्य। और यह पुर्ड, थैंक्य। काई तलग रहे पात्वी है? मैंने अपको चाय दी तो अपने मुझे थैंक्यो पोला पाता, तो मेरे तो कित... नी तो तो तो पुल पाके। इसलिया अभी पोल रही हो, तैंक्य। सड़ड़ड़ड़ड़ड़ड़ड़ड़ड़ड़ड़ड़ड़ड़ड़ड़ड़ड़ड़ड़ड़ड़ड़ड़ड़ड़ड़ड़ड़ड़ड़ड़ड़ड़ड़ड़ फॉरीःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःःः अप्रटर हाथी कभी आ गया वरी गोड़ शेवर का भी आ गया जेटाला...ः इस आति का मेंटिनस का चे कभी भी ताइपड़ नहीं आता देगालो फोंग करा उठवर गरा


In [4]:
!pip install faster-whisper jiwer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 52.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.3/35.3 MB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 105.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 86.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 6.2 MB/s eta 0:00:00


In [1]:
from faster_whisper import WhisperModel
import jiwer

# Initialize Whisper model
model_size = "large-v3"
model = WhisperModel(model_size, device="cuda", compute_type="float16")

# Transcribe audio
segments, info = model.transcribe("/content/ted_60.wav", beam_size=5)

# Display detected language
print("Detected language '%s' with probability %f" % (info.language, info.language_probability))

# Extract transcribed text
transcribed_text = " ".join([segment.text for segment in segments])

# Ground truth (replace this with actual correct transcript)
ground_truth = """
So in college, I was a government major, which means I had to write a lot of papers. Now, when a normal student writes a paper, they might spread the work out a little like this. So you know.
You get started maybe a little slowly, but you get enough done in the first week that with some heavier days later on, everything gets done and things stay civil. And I would want to do that, like that. That would be the plan. I would have it all ready to go, but then actually the paper would come along, and then I would kind of do this.
And that would happen to every single paper. But then came my 90-page senior thesis, a paper you're supposed to spend a year on. I knew for a paper like that, my normal workflow was not an option, it was way too big a project. So I planned things out, and I decided it kind of had to go something like this. This is how the year would go. So I'd start off light, and I'd bump it up.
"""

# Calculate WER
wer = jiwer.wer(ground_truth, transcribed_text)

print("\nTranscribed Text:\n", transcribed_text)
print("\nWord Error Rate (WER):", wer)


Detected language 'en' with probability 0.997559

Transcribed Text:
  So in college, I was a government major,  which means I had to write a lot of papers.  Now, when a normal student writes a paper,  they might spread the work out a little like this.  So, you know, you get started maybe a little slowly,  but you get enough done in the first week  that with some heavier days later on,  everything gets done and things stay civil.  And I would want to do that like that.  That would be the plan.  I would have it all ready to go,  but then actually the paper would come along,  and then I would kind of do this.  And that would happen to every single paper.  But then came my 90-page senior thesis,  a paper you're supposed to spend a year on.  And I knew for a paper like that,  my normal workflow was not an option.  It was way too big a project.  So I planned things out, and I decided  I kind of had to go something like this.  This is how the year would go.  So I'd start off light,  and I'd b

In [2]:
from faster_whisper import WhisperModel

model_size = "large-v3"

# Run on GPU with FP16
model = WhisperModel(model_size, device="cuda", compute_type="float16")

# or run on GPU with INT8
# model = WhisperModel(model_size, device="cuda", compute_type="int8_float16")
# or run on CPU with INT8
# model = WhisperModel(model_size, device="cpu", compute_type="int8")

segments, info = model.transcribe("/content/tmkoc.mp3", beam_size=5)

print("Detected language '%s' with probability %f" % (info.language, info.language_probability))

for segment in segments:
    print("[%.2fs -> %.2fs] %s" % (segment.start, segment.end, segment.text))

Detected language 'hi' with probability 0.772949
[0.00s -> 1.18s]  उसके लिए थैंक यू
[1.18s -> 3.06s]  कभी-कभी आप पापड मिल देते हो
[3.06s -> 4.06s]  उसके लिए भी थैंक यू
[4.06s -> 7.46s]  दो मेले पहले आपने मेरे चप्पल सीला के लेके आये थे
[7.46s -> 8.46s]  उसके लिए भी थैंक यू
[8.46s -> 11.06s]  आप मेरी सारी को भी हिस्सी कर देते हो
[11.06s -> 12.96s]  उसके लिए भी थैंक यू
[12.96s -> 16.34s]  और एक दिन दो थैंक यू पूरा का चलने बात रही है
[16.34s -> 18.76s]  मैंने आपको जब चाय दी तो आपने मुझे थैंक यू बोला था
[18.76s -> 21.32s]  तो मेरे तो किसने थैंक यू बोलने बाखे
[21.32s -> 24.82s]  इसलिए अभी बोल रही हूँ थैंक यू
[24.82s -> 26.26s]  सौरी
[26.26s -> 30.62s]  परसो चाय में शक्कर जादा हो गई थी उसके लिए सौरी
[30.62s -> 33.80s]  खादे में नबक जादा हो गई था उसके लिए भी सौरी
[33.80s -> 36.94s]  और आपके गुर्टे का पटन लगाना भूल गई थी उसके लिए भी सौरी
[36.94s -> 39.12s]  एक गुर्टे का पटन लगाना भूल गई थी उसके लिए भी सौरी
[39.12s -> 42.36s]  आहो पति पति में थैंक यू और सोरी होता ही नहीं है
[42.36s -> 43.84s]  क

In [3]:
!pip install insanely-fast-whisper

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 kB 5.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 898.7/898.7 kB 54.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 819.0/819.0 kB 53.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.5/58.5 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.1/48.1 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.4/51.4 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.9/125.9 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 864.1/864.1 kB 43.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 86.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
!pip install sentence-transformers scipy

In [6]:
import threading
from faster_whisper import WhisperModel
from sentence_transformers import SentenceTransformer
from scipy.spatial.distance import cosine

# Initialize models
whisper_model = WhisperModel("large-v3", device="cuda", compute_type="float16")
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')  # Light and fast

# Transcription function
def transcribe_audio(audio_path, results, key):
    print(f"Transcribing {audio_path}...")
    segments, info = whisper_model.transcribe(audio_path, beam_size=5)
    transcript = " ".join([seg.text for seg in segments])
    results[key] = {
        "language": info.language,
        "language_probability": info.language_probability,
        "transcript": transcript
    }

# Input audio paths
audio_file_1 = "/content/tmkoc_part1.mp3"
audio_file_2 = "/content/tmkoc_part2.mp3"

# Store transcripts
transcriptions = {}

# Start transcription threads
t1 = threading.Thread(target=transcribe_audio, args=(audio_file_1, transcriptions, "file1"))
t2 = threading.Thread(target=transcribe_audio, args=(audio_file_2, transcriptions, "file2"))

t1.start()
t2.start()
t1.join()
t2.join()

# Extract transcripts
transcript1 = transcriptions["file1"]["transcript"]
transcript2 = transcriptions["file2"]["transcript"]

# Generate embeddings
embedding1 = embedding_model.encode(transcript1)
embedding2 = embedding_model.encode(transcript2)

# Calculate cosine similarity
similarity_score = 1 - cosine(embedding1, embedding2)

# Output
print("\n--- Cosine Similarity Results ---")
print(f"Transcript 1 Length: {len(transcript1)} characters")
print(f"Transcript 2 Length: {len(transcript2)} characters")
print(f"Cosine Similarity: {similarity_score:.4f}")


Transcribing /content/tmkoc_part1.mp3...
Transcribing /content/tmkoc_part2.mp3...

--- Cosine Similarity Results ---
Transcript 1 Length: 293 characters
Transcript 2 Length: 328 characters
Cosine Similarity: 0.8858
